In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("cleaned_data.csv")

In [7]:
df.head()

,genres,title,price,id,developer,release_year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,0.00,643980,Secret Level SRL,2018
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,0.99,767400,彼岸领域,2017
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,2018


In [3]:
#Ordeno las columnas
ordered_columns = ["title","id","developer","release_year","genres","price"]
df = df[ordered_columns]

In [4]:
#Hago el encoding de las columnas de género para hacer el sistema.
import ast
df["genres"] = df["genres"].apply(lambda x: ast.literal_eval(x))
genres = pd.DataFrame(df["genres"].tolist())
genres_obj = genres.stack()
genres_df = pd.get_dummies(genres_obj)
genres_df = genres_df.groupby(level = [0], axis = 0).sum()


In [5]:
genres_df.sum().sort_values(ascending=False)

Indie                        15123
Action                       10697
Casual                        8061
Adventure                     7809
Strategy                      6701
Simulation                    6512
RPG                           5121
Free to Play                  1881
Early Access                  1383
Sports                        1206
Racing                        1049
Massively Multiplayer         1000
Design &amp; Illustration      450
Utilities                      319
Web Publishing                 260
Animation &amp; Modeling       170
Education                      121
Video Production               104
Software Training              102
Audio Production                85
Photo Editing                   77
Accounting                       7
dtype: int64

In [6]:
genres_df = genres_df[["Indie","Action","Casual","Adventure","Strategy","Simulation","RPG","Free to Play","Sports","Racing"]]

In [7]:
df = pd.concat([df, genres_df], axis = 1)
df = df.drop(columns="genres")

In [10]:
df.columns

Index(['title', 'id', 'developer', 'release_year', 'price', 'Indie', 'Action',
       'Casual', 'Adventure', 'Strategy', 'Simulation', 'RPG', 'Free to Play',
       'Sports', 'Racing'],
      dtype='object')

In [11]:
#Selecciono las primeras 5000 filas para que mi pc de 8gb de ram pueda procesarlo.

first_5000_rows = df.head(5000)
first_5000_rows.to_csv("first_20000_rows.csv", index=False)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

#Selecciono las columnas para hacer la matriz del coseno de similiaridad.
features = first_5000_rows[['Indie', 'Action', 'Casual', 'Adventure', 'Strategy', 'Simulation', 'RPG', 'Free to Play',
       'Sports', 'Racing']]

similarity_matrix = cosine_similarity(features)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

def calcular_recomendacion(first_5000_rows, title, top_n=5):
    #Selecciono las columnas para hacer la matriz del coseno de similiaridad.
    features = first_5000_rows[['Indie', 'Action', 'Casual', 'Adventure', 'Strategy', 'Simulation', 'RPG', 'Free to Play', 'Sports', 'Racing']]

    similarity_matrix = cosine_similarity(features)

    indice_juego = first_5000_rows[first_5000_rows['title'] == title].index[0]

    # Calculo la similitud del coseno para ese juego
    similar_juegos = list(enumerate(similarity_matrix[indice_juego]))

    # Ordeno las recomendaciones
    similar_juegos = sorted(similar_juegos, key=lambda x: x[1], reverse=True)

    # Selecciono las n primeras posiciones
    top_n_similar_juegos = similar_juegos[1:top_n + 1]

    # Selecciono sus indices
    top_n_indices = [x[0] for x in top_n_similar_juegos]

    # Get the titles of the top N similar games
    juegos_recomendados = first_5000_rows.iloc[top_n_indices]['title'].tolist()

    return juegos_recomendados

In [16]:
juegos = first_5000_rows["title"]

# Create a dictionary to store recommendations
recomendaciones_dict = {}

for title in juegos:
    recomendaciones = calcular_recomendacion(first_5000_rows, title, top_n=5)
    recomendaciones_dict[titulo] = recomendaciones

# Convierto el diccionario en un dataframe
recomendaciones_df = pd.DataFrame(recomendaciones_dict.items(), columns=["Game", "Recommended Games"])


In [18]:
recomendaciones_df.to_csv("Recommendation_system.csv",index=False)

In [19]:
recomendaciones_df

,Game,Recommended Games
0,Lost Summoner Kitty,"[Pixel Puzzles 2: Anime, World of Cinema - Dir..."
1,Ironbound,"[Shadow Hunter, Immortal Empire, Tactical Craf..."
2,Real Pool 3D - Poolians,"[SNOW - All Access Basic Pass, SNOW - All Acce..."
3,弹炸人2222,"[Atomic Adam: Episode 1, Biozone, Luxor: 5th P..."
4,Battle Royale Trainer,"[The Tomorrow War, Beyond Space Remastered Edi..."
...,...,...
4995,Krautscape,"[SkyDrift, Spectra, SinaRun, Krautscape, Carma..."
4996,The Kindred,"[Gratuitous Space Battles, Gratuitous Space Ba..."
4997,Disgaea PC / 魔界戦記ディスガイア PC,"[Rise of the Argonauts, Drakensang, Guild Wars..."
4998,24 Hours 'til Rescue: Cheat Pack!,"[Fearless Fantasy, Luna: Shattered Hearts: Epi..."
